In [ ]:
import json
import ipywidgets as widgets
from IPython.display import display, Markdown
import openai

class StructuredLLMAgent:
    def __init__(self):
        # Initialize widgets
        self.prompt_input = widgets.Textarea(
            value="Create a list of 5 programming languages with their release year, paradigm, and main use cases",
            placeholder="Enter your prompt here...",
            description='Prompt:',
            layout=widgets.Layout(width='90%', height='100px')
        )

        self.structure_selector = widgets.Dropdown(
            options=['JSON', 'XML', 'CSV', 'Markdown Table', 'YAML'],
            value='JSON',
            description='Format:'
        )

        self.schema_input = widgets.Textarea(
            value='{"languages": [{"name": "string", "year": "number", "paradigm": "string", "use_cases": "array"}]}',
            placeholder='Optional: Define your schema (e.g., {"name": "string", "age": "number"})',
            description='Schema:',
            layout=widgets.Layout(width='90%', height='80px')
        )

        self.generate_btn = widgets.Button(
            description="Generate Response",
            button_style='primary'
        )
        self.generate_btn.on_click(self.generate_structured_response)

        self.output_area = widgets.Output()

        # Assemble UI
        self.agent_panel = widgets.VBox([
            widgets.HTML("<h2>Structured LLM Agent</h2>"),
            self.prompt_input,
            widgets.HBox([self.structure_selector, self.schema_input]),
            self.generate_btn,
            widgets.HTML("<hr>"),
            widgets.HTML("<b>Structured Output:</b>"),
            self.output_area
        ])

    def get_structured_prompt(self, user_prompt, format_type, schema):
        """Create a system prompt that enforces structured output"""

        format_instructions = {
            'JSON': "Respond with a valid JSON object. Ensure proper syntax with double quotes for keys and string values.",
            'XML': "Respond with well-formed XML. Include proper opening and closing tags.",
            'CSV': "Respond with CSV format. Include headers and ensure proper comma separation.",
            'Markdown Table': "Respond with a Markdown table using pipe symbols. Include header row and separator.",
            'YAML': "Respond with valid YAML format. Use proper indentation and key-value pairs."
        }

        system_prompt = f"""You are a structured data generator. Your response must be in {format_type} format.

{format_instructions[format_type]}

"""

        if schema and schema.strip():
            try:
                schema_obj = json.loads(schema)
                system_prompt += f"\nRequired schema: {json.dumps(schema_obj, indent=2)}"
            except:
                system_prompt += f"\nSchema provided but not valid JSON. Please follow: {schema}"

        system_prompt += f"\n\nUser query: {user_prompt}"

        return system_prompt

    def generate_structured_response(self, b):
        self.output_area.clear_output()

        user_prompt = self.prompt_input.value.strip()
        if not user_prompt:
            with self.output_area:
                print("Please enter a prompt")
            return

        format_type = self.structure_selector.value
        schema = self.schema_input.value

        # For demonstration, using a mock LLM response
        structured_prompt = self.get_structured_prompt(user_prompt, format_type, schema)

        with self.output_area:
            print("Generated Structured Prompt:")
            print("=" * 50)
            print(structured_prompt)
            print("\n" + "=" * 50)
            print("\nMock LLM Response (structured):")

            # Generate context-aware mock responses
            mock_response = self.get_context_aware_response(user_prompt, format_type, schema)

            if format_type == 'JSON':
                try:
                    formatted_json = json.dumps(json.loads(mock_response), indent=2)
                    display(Markdown(f"```json\n{formatted_json}\n```"))
                except:
                    display(Markdown(f"```json\n{mock_response}\n```"))
            elif format_type == 'Markdown Table':
                display(Markdown(mock_response))
            else:
                display(Markdown(f"```{format_type.lower()}\n{mock_response}\n```"))

    def get_context_aware_response(self, prompt, format_type, schema):
        """Generate context-aware mock responses based on the user prompt"""

        prompt_lower = prompt.lower()

        # Programming languages response
        if any(keyword in prompt_lower for keyword in ['programming language', 'programming', 'language', 'python', 'java', 'javascript']):
            return self.get_programming_languages_response(format_type)

        # Countries response
        elif any(keyword in prompt_lower for keyword in ['country', 'countries', 'capital', 'population']):
            return self.get_countries_response(format_type)

        # Product comparison response
        elif any(keyword in prompt_lower for keyword in ['compare', 'comparison', 'vs', 'versus']):
            return self.get_comparison_response(prompt, format_type)

        # Book list response
        elif any(keyword in prompt_lower for keyword in ['book', 'books', 'author', 'publication']):
            return self.get_books_response(format_type)

        # Default response for other queries
        else:
            return self.get_general_structured_response(prompt, format_type)

    def get_programming_languages_response(self, format_type):
        """Generate programming languages data in requested format"""

        languages_data = {
            "languages": [
                {
                    "name": "Python",
                    "year": 1991,
                    "paradigm": "Multi-paradigm",
                    "use_cases": ["Web Development", "Data Science", "AI/ML", "Automation"]
                },
                {
                    "name": "JavaScript",
                    "year": 1995,
                    "paradigm": "Multi-paradigm",
                    "use_cases": ["Web Development", "Frontend", "Backend", "Mobile Apps"]
                },
                {
                    "name": "Java",
                    "year": 1995,
                    "paradigm": "Object-Oriented",
                    "use_cases": ["Enterprise Applications", "Android Development", "Web Services"]
                },
                {
                    "name": "C++",
                    "year": 1985,
                    "paradigm": "Multi-paradigm",
                    "use_cases": ["System Programming", "Game Development", "High-Performance Computing"]
                },
                {
                    "name": "Go",
                    "year": 2012,
                    "paradigm": "Concurrent",
                    "use_cases": ["Backend Services", "Cloud Applications", "CLI Tools"]
                }
            ]
        }

        return self.format_data(languages_data, format_type, "Programming Languages")

    def get_countries_response(self, format_type):
        """Generate countries data in requested format"""

        countries_data = {
            "countries": [
                {"name": "United States", "capital": "Washington D.C.", "population": 331000000, "continent": "North America"},
                {"name": "Germany", "capital": "Berlin", "population": 83000000, "continent": "Europe"},
                {"name": "Japan", "capital": "Tokyo", "population": 126000000, "continent": "Asia"},
                {"name": "Brazil", "capital": "Brasília", "population": 213000000, "continent": "South America"},
                {"name": "Egypt", "capital": "Cairo", "population": 104000000, "continent": "Africa"}
            ]
        }

        return self.format_data(countries_data, format_type, "Countries Information")

    def get_books_response(self, format_type):
        """Generate books data in requested format"""

        books_data = {
            "books": [
                {"title": "The Great Gatsby", "author": "F. Scott Fitzgerald", "year": 1925, "genre": "Fiction"},
                {"title": "1984", "author": "George Orwell", "year": 1949, "genre": "Dystopian"},
                {"title": "To Kill a Mockingbird", "author": "Harper Lee", "year": 1960, "genre": "Fiction"},
                {"title": "Pride and Prejudice", "author": "Jane Austen", "year": 1813, "genre": "Romance"},
                {"title": "The Hobbit", "author": "J.R.R. Tolkien", "year": 1937, "genre": "Fantasy"}
            ]
        }

        return self.format_data(books_data, format_type, "Book Collection")

    def get_comparison_response(self, prompt, format_type):
        """Generate comparison data based on the prompt"""

        if 'cloud' in prompt.lower():
            comparison_data = {
                "cloud_providers": [
                    {"provider": "AWS", "free_tier": "12 months", "popular_services": ["EC2", "S3", "Lambda"], "pricing_model": "Pay-as-you-go"},
                    {"provider": "Azure", "free_tier": "12 months", "popular_services": ["Virtual Machines", "Blob Storage", "Functions"], "pricing_model": "Pay-as-you-go"},
                    {"provider": "GCP", "free_tier": "Always free tier", "popular_services": ["Compute Engine", "Cloud Storage", "Cloud Functions"], "pricing_model": "Pay-as-you-go"}
                ]
            }
            return self.format_data(comparison_data, format_type, "Cloud Providers Comparison")

        else:
            # Generic comparison
            comparison_data = {
                "comparison": [
                    {"item": "Option A", "features": ["Feature 1", "Feature 2"], "price": "$100", "rating": 4.5},
                    {"item": "Option B", "features": ["Feature 1", "Feature 3"], "price": "$150", "rating": 4.2},
                    {"item": "Option C", "features": ["Feature 2", "Feature 4"], "price": "$120", "rating": 4.7}
                ]
            }
            return self.format_data(comparison_data, format_type, "Comparison Analysis")

    def get_general_structured_response(self, prompt, format_type):
        """Generate general structured response for unknown topics"""

        general_data = {
            "response": {
                "query": prompt,
                "results": [
                    {"item": "Result 1", "description": "Description for result 1", "category": "General"},
                    {"item": "Result 2", "description": "Description for result 2", "category": "General"},
                    {"item": "Result 3", "description": "Description for result 3", "category": "General"}
                ],
                "summary": f"Structured response for: {prompt}"
            }
        }

        return self.format_data(general_data, format_type, "General Response")

    def format_data(self, data, format_type, title):
        """Format data according to the requested format type"""

        if format_type == 'JSON':
            return json.dumps(data, indent=2)

        elif format_type == 'XML':
            return self.dict_to_xml(data)

        elif format_type == 'CSV':
            return self.dict_to_csv(data)

        elif format_type == 'Markdown Table':
            return self.dict_to_markdown_table(data, title)

        elif format_type == 'YAML':
            return self.dict_to_yaml(data)

        return json.dumps(data, indent=2)

    def dict_to_xml(self, data):
        """Convert dictionary to simple XML format"""
        def dict_to_xml_str(d, root_name='root'):
            parts = [f'<{root_name}>']
            for key, value in d.items():
                if isinstance(value, list):
                    for item in value:
                        if isinstance(item, dict):
                            parts.append(dict_to_xml_str(item, key))
                        else:
                            parts.append(f'<{key}>{item}</{key}>')
                elif isinstance(value, dict):
                    parts.append(dict_to_xml_str(value, key))
                else:
                    parts.append(f'<{key}>{value}</{key}>')
            parts.append(f'</{root_name}>')
            return '\n'.join(parts)

        return '<?xml version="1.0" encoding="UTF-8"?>\n' + dict_to_xml_str(data, 'response')

    def dict_to_csv(self, data):
        """Convert dictionary to CSV format"""
        # Simple CSV conversion for the first list found
        for key, value in data.items():
            if isinstance(value, list) and value:
                headers = value[0].keys()
                csv_lines = [','.join(headers)]
                for item in value:
                    csv_lines.append(','.join(str(item.get(h, '')) for h in headers))
                return '\n'.join(csv_lines)
        return "No tabular data found"

    def dict_to_markdown_table(self, data, title):
        """Convert dictionary to Markdown table"""
        md = f"## {title}\n\n"

        for key, value in data.items():
            if isinstance(value, list) and value:
                headers = value[0].keys()
                md += f"### {key.replace('_', ' ').title()}\n\n"
                md += '| ' + ' | '.join(str(h).replace('_', ' ').title() for h in headers) + ' |\n'
                md += '|' + '|'.join(['---'] * len(headers)) + '|\n'
                for item in value:
                    md += '| ' + ' | '.join(str(item.get(h, '')) for h in headers) + ' |\n'
                md += '\n'

        return md

    def dict_to_yaml(self, data):
        """Convert dictionary to YAML format"""
        def dict_to_yaml_str(d, indent=0):
            yaml_str = ""
            space = "  " * indent
            if isinstance(d, dict):
                for key, value in d.items():
                    if isinstance(value, (dict, list)):
                        yaml_str += f"{space}{key}:\n"
                        yaml_str += dict_to_yaml_str(value, indent + 1)
                    else:
                        yaml_str += f"{space}{key}: {value}\n"
            elif isinstance(d, list):
                for item in d:
                    if isinstance(item, (dict, list)):
                        yaml_str += f"{space}-\n"
                        yaml_str += dict_to_yaml_str(item, indent + 1)
                    else:
                        yaml_str += f"{space}- {item}\n"
            return yaml_str

        return dict_to_yaml_str(data)

# Create and display Task 1 agent
task1_agent = StructuredLLMAgent()
display(task1_agent.agent_panel)